# Compare models in a hybrid setup: Motion feedback- Force actuation

- numerical model: simulate the floater dynamics.
- physical turbine: measure forces and accelerations
- Ml model: multi-output regressor

In [1]:
from fowt_ml.pipeline import Pipeline

In [2]:
example_config_file = "../../src/example_config.yml"

In [3]:
my_pipeline = Pipeline(example_config_file)

In [4]:
# set correct path for mat file: add the correct path
my_pipeline.data_config["exp699"]["path_file"] = "/home/sarah/temp/hybridlabs/data_example/exp699.mat"

In [5]:
# check the targets and predictors that are provided in the comfig
print("targets are:", my_pipeline.target_labels)

targets are: ['pos_act6[0]', 'pos_act6[1]', 'pos_act6[2]', 'pos_act6[3]', 'pos_act6[4]', 'pos_act6[5]']


In [6]:
print("features are:", my_pipeline.predictors_labels)

features are: ['force_tt_meas6[0]', 'force_tt_meas6[1]', 'force_tt_meas6[2]', 'force_tt_meas6[3]', 'force_tt_meas6[4]', 'force_tt_meas6[5]', 'acc_tb_meas3[0]', 'acc_tb_meas3[1]', 'acc_tb_meas3[2]', 'acc_tt_meas3[0]', 'acc_tt_meas3[1]', 'acc_tt_meas3[2]', 'spd_rot_act', 'wind_speed']


In [ ]:
# check the models and metrics that are provided by the config
print(my_pipeline.model_names)

In [ ]:
print(my_pipeline.metric_names)

{'ElasticNetRegression': {}, 'LeastAngleRegression': {}, 'LassoRegression': {}, 'LinearRegression': {}, 'SklearnGPRegressor': {'num_inducing': 100, 'num_latents': 3, 'num_epochs': 10}, 'RandomForest': {'n_estimators': 50, 'max_depth': 5, 'bootstrap': True, 'max_samples': 10000}, 'MultilayerPerceptron': {'hidden_layer_sizes': 10, 'max_iter': 50}, 'XGBoostRegression': {'n_estimators': 10, 'max_depth': 10, 'tree_method': 'hist'}}
['neg_mean_absolute_error', 'neg_root_mean_squared_error', 'r2', 'model_fit_time', 'model_predict_time']


In [8]:
# setup the pipeline
my_pipeline.setup(data="exp699")

In [9]:
# compare models on test data
models, scores = my_pipeline.compare_models(sort="model_fit_time")

2025/10/23 11:05:03 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'fowt-ml'}
INFO:fowt_ml.pipeline:Saving grid scores to grid_scores.csv
INFO:fowt_ml.pipeline:Saving best model to joblib format in best_model.joblib


In [10]:
scores

,neg_mean_absolute_error,neg_root_mean_squared_error,r2,model_fit_time,model_predict_time
LeastAngleRegression,-1.603053,-2.130915,0.107324,0.108,0.001
ElasticNetRegression,-1.633376,-2.164626,0.010878,0.150,0.001
LassoRegression,-1.634780,-2.166315,-0.000018,0.177,0.001
LinearRegression,-1.603053,-2.130915,0.107324,0.184,0.001
XGBoostRegression,-1.581722,-2.086596,0.153208,2.554,0.001
RandomForest,-1.600478,-2.122782,0.106647,4.160,0.002
MultilayerPerceptron,-1.598176,-2.118952,0.120684,6.375,0.001
SklearnGPRegressor,-1.598109,-2.121805,0.123206,103.987,0.003


In [11]:
models

{'ElasticNetRegression': TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                       StandardScaler()),
                                                      ('model', ElasticNet())]),
                            transformer=StandardScaler()),
 'LeastAngleRegression': TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                       StandardScaler()),
                                                      ('model', Lars())]),
                            transformer=StandardScaler()),
 'LassoRegression': TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                                                       StandardScaler()),
                                                      ('model', Lasso())]),
                            transformer=StandardScaler()),
 'LinearRegression': TransformedTargetRegressor(regressor=Pipeline(steps=[('scaler',
                      

In [12]:
# compare models using cross_validation
models, scores = my_pipeline.compare_models(sort="model_predict_time", cross_validation=True)

INFO:fowt_ml.pipeline:Logging experiment to MLflow with id 542351623230069078
2025/10/23 11:11:51 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'fowt-ml'}
INFO:fowt_ml.pipeline:Saving grid scores to grid_scores.csv
INFO:fowt_ml.pipeline:Saving best model to joblib format in best_model.joblib


In [13]:
scores

,model_fit_time,neg_mean_absolute_error,neg_root_mean_squared_error,r2,model_predict_time
LeastAngleRegression,0.114667,-1.607105,-2.138814,0.108090,0.001000
LinearRegression,0.151000,-1.607105,-2.138814,0.108090,0.001000
MultilayerPerceptron,5.909000,-1.602276,-2.126795,0.121668,0.001000
XGBoostRegression,1.999333,-1.590279,-2.101709,0.146525,0.001333
ElasticNetRegression,0.162667,-1.637038,-2.172412,0.011023,0.001667
LassoRegression,0.394000,-1.638445,-2.174126,-0.000018,0.002667
RandomForest,4.104667,-1.604850,-2.131100,0.107288,0.002667
SklearnGPRegressor,70.743667,-1.602289,-2.130473,0.123055,0.003333
